In [463]:
# Se importa la libreria random para hacer uso de números aleatorios, y una libreria para guardar listas
import random
from collections import defaultdict, deque

In [464]:
class Nodo: # Clase nodo guarda el nombre de cada nodo en forma de string (1)
    def __init__(self, valor):
        self.valor = valor
        self.vecinos = []
    # Se crea un metodo para agregar vecinos al arbol generado utilizado en BFS y DFS
    def agregar_vecino(self, vecino):
        self.vecinos.append(vecino)
    # El nodo se guarda con el formato dado desde la función
    def __str__(self):
        return str(self.valor)

class Arista: # Clase arista guarda el nombre de cada arista en forma de string ( 1 -> 2 )
    def __init__(self, origen, destino):
        self.origen = origen
        self.destino = destino
    # La arista se guarda con el formato 1 -> 2
    def __str__(self):
        return f"{self.origen} -> {self.destino}"

class Grafo:
    def __init__(self,nombre_archivo):
        # Se inicializa el arreglo de nodos y aristas del grafo
        self.nodos = {}
        self.aristas = {}
        self.nombre_archivo = nombre_archivo
    
    # Se guarda cada nodo a una lista de nodos
    def agregar_nodo(self, valor):
        if valor not in self.nodos:
            nodo = Nodo(valor)
            self.nodos[valor] = nodo
    # Se guarda cada arista a una lista de aristas
    def agregar_arista(self, valor_origen, valor_destino):
        #if valor_origen not in self.nodos and valor_destino not in self.nodos:
        # Se verifica que el nodo de origen y destino esten en la lista de nodos
        if valor_origen in self.nodos and valor_destino in self.nodos:
            self.agregar_nodo(valor_origen)
            self.agregar_nodo(valor_destino)
            nodo_origen = self.nodos[valor_origen]
            nodo_destino = self.nodos[valor_destino]
            # Se agrega el nodo vecino para BFS y DFS
            nodo_origen.agregar_vecino(nodo_destino)
        arista = Arista(valor_origen, valor_destino)
        if valor_origen not in self.aristas:
            self.aristas[valor_origen] = []
        if arista not in self.aristas:
            self.aristas[valor_origen].append(arista)

    def __str__(self):
        # Se guardan los valores de los nodos y aristas con el formato:
        """
        digraph G {
            // Nodos
            1;
            2;
            3;
            // Aristas
            1 -> 2;
            1 -> 3;
            2 -> 3;
        }
        """
        nodos = ", ".join(str(nodo) for nodo in self.nodos.values())
        aristas = ";\n\t".join(str(arista) for aristas_nodo in self.aristas.values() for arista in aristas_nodo)
        aristas = "{\n\t" + aristas + ";\n}"
        # Se guarda la cadena de texto como un archivo gv
        with open(self.nombre_archivo, "w") as archivo:
            archivo.write(f"digraph G {aristas}")
        return f"digraph G {aristas}"
    
    # Se define el metodo para el algoritmo BFS
    def bfs(self, valor_inicio):
        # Se verifica que el nodo de inicio se encuentre en el grafo
        if valor_inicio not in self.nodos:
            return
        visitado = set() # Se guardan los nodos visitados en el conjunto generado
        cola = deque([self.nodos[valor_inicio]]) # Se define una "cola" con los nodos visitados
        padres = {self.nodos[valor_inicio]: None} # Se guardan los nodos padres de cada arista para obtener el arbol BFS
        while cola: # Se establece la condicion de paro del algoritmo cando no hay mas elementos en la cola
            nodo_actual = cola.popleft()
            if nodo_actual not in visitado: # Se establece si el nodo actual no se ha visitado
                #print(nodo_actual.valor, end=" ")
                visitado.add(nodo_actual) # Se marca el nodo como visitado
                for vecino in nodo_actual.vecinos:
                    if vecino not in visitado:
                        cola.append(vecino) # Se agrega el nodo vecino a la cola
                        padres[vecino] = nodo_actual # Se guarda el nodo de origen o nodo padre para cada arista
        arbol_bfs = {} # Se inicializa un diccionario con los elementos del arbol generado por BFS
        for nodo, padre in padres.items():
            if padre is not None:
                # Se establecen los nodos y aristas del arbol generado
                arbol_bfs.setdefault(padre.valor, []).append(nodo.valor)
        return arbol_bfs
    
    # Se define el metodo para el algoritmo DFS
    def dfs(self, valor_inicio):
        # Se verifica que el nodo de inicio se encuentre en el grafo
        if valor_inicio not in self.nodos:
            return
        visitado = set() # Se guardan los nodos visitados en el conjunto generado
        padres = {self.nodos[valor_inicio]: None}  # Se guardan los nodos padres de cada arista para obtener el arbol 
        def dfs_recursivo(nodo_actual): # Se define una funcion con el algoritmo de forma iterativa
            #print(nodo_actual.valor, end=" ")
            visitado.add(nodo_actual) # Se marca el nodo como visitado
            for vecino in nodo_actual.vecinos:
                if vecino not in visitado:
                    dfs_recursivo(vecino)
                    padres[vecino] = nodo_actual # Se guarda el nodo de origen o nodo padre para cada arista
        dfs_recursivo(self.nodos[valor_inicio]) # Se llama al algoritmo iterativo
        arbol_dfs = {} # Se inicializa un diccionario con los elementos del arbol generado por BFS
        for hijo, padre in padres.items():
            if padre is not None:
                # Se establecen los nodos y aristas del arbol generado
                arbol_dfs.setdefault(padre, []).append(hijo)
        return arbol_dfs

In [465]:
# Se define una funcion para escribir el arbol generado en un archivo gv
def escribir_arbol(arbol_bfs,nombre_archivo):
    print("digraph G {")
    with open(nombre_archivo, "w") as archivo:
        archivo.write("digraph G {\n")
        for padre, hijos in arbol_bfs.items():
            for hijo in hijos:
                archivo.write(f"\t{padre} -> {hijo};\n")
                print(f"\t{padre} -> {hijo};")
        archivo.write("}")
    print("}")

In [466]:
def randomErdos(n,m,p):
    nombre_archivo = "Erdos_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan n nodos al grafo
    for i in range(n):
        g.agregar_nodo((i))
    # se conectan m nodos de forma aleatoria. (n>m)
    for i in range(m):
        u = random.randint(0,n-1)
        v = random.randint(0,n-1)
        if u != v: # Se omiten las conexiones hacia un mismo nodo
            g.agregar_arista((u),(v))
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    return g

In [467]:
def Malla(n,m,p):
    nombre_archivo = "Malla_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan n*m nodos al grafo
    for i in range(n*m):
        g.agregar_nodo((i+1))
    # Se conectan los nodos de la malla i,j+1 y i+1,j
    for i in range(n):
        for j in range(m):
            # La malla se recorre de la siguiente forma:
            """
            1  2  3
            4  5  6
            7  8  9
            """
            u = i*n+j+1
            v1 = i*n+j+2
            v2 = (i+1)*n+j+1
            # Se descartan todas las posibles combinaciones cuyos nodos superen el valor total de nodos en el grafo
            if v1 > n*m or v2 > n*m:
                v2 = u
            if u != v1: # Se omiten las conexiones hacia un mismo nodo
                g.agregar_arista((u),(v1))
            if u != v2: # Se omiten las conexiones hacia un mismo nodo
                g.agregar_arista((u),(v2))
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    return g

In [468]:
def Geografico(n,r,p):
    nombre_archivo = "Geografico_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    x = []; y = []
    # Se agregan n nodos al grafo
    for i in range(n):
        g.agregar_nodo((i))
        # se define una posición aleatria del nodo en el espacio
        x.append(random.randint(0,5*r))
        y.append(random.randint(0,5*r))
    for i in range(n):
        for j in range(n):
            dist = (x[i]-x[j])**2+(y[i]-y[j])**2 # Se calcula la distancia entre los nodos del grafo
            # Se conectan los nodos que se encuentran dentro del rango de distancia dado
            if dist < r and i != j: # Se omiten las conexiones hacia un mismo nodo
                g.agregar_arista((i),(j))
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    return g

In [469]:
def Gilbert(n,p,x):
    nombre_archivo = "Gilbert_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan n nodos al grafo
    for i in range(n):
        g.agregar_nodo((i))
    for i in range(n):
        lista = [] # Se guardan los nodos previamente conectados para evitar que se dupliquen las aristas
        for j in range(n):
            # Se conectan 2 nodos de forma aleatoria con una probabilidad p
            cond = j in lista # Se determina si el nodo se encuentra en el bucle actual para evitar repeticiones
            if random.random() <= p and i != j and cond == False: # Se omiten las conexiones hacia un mismo nodo
                g.agregar_arista((i),(j))
            lista.append(j) # Se guardan los nodos actualmente conectados para evitar que se dupliquen las aristas
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(x)
    arbol_dfs = g.dfs(x)
    escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    return g

In [470]:
def Barabasi(n,m,p):
    nombre_archivo = "Barabasi_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan m nodos al grafo
    for i in range(1,m):
        g.agregar_nodo((i))
        # Se conectan los m nodos agregados
        for j in range(1,i):
            g.agregar_arista((i),(j))
    # Se determina la preferencia de conexión de los nodos añadidos
    preferencia = [nodo for nodo in g.nodos.keys() for _ in range(len(g.nodos))]
    for i in range(m, n):
        lista = [] # Se guardan los nodos previamente conectados para evitar que se dupliquen las aristas
        g.agregar_nodo(i) # Se agrega un nuevo nodo
        conexiones = random.sample(preferencia, m) # Se conecta el nuevo nodo al grafo
        for nodo in conexiones:
            cond = nodo in lista # Se determina si el nodo se encuentra en el bucle actual para evitar repeticiones
            if i != nodo and cond == False: # Se omiten las conexiones hacia un mismo nodo y las aristas repetidas
                g.agregar_arista(i, nodo)
                preferencia.extend([nodo, i])
            lista.append(nodo) # Se guardan los nodos actualmente conectados para evitar que se dupliquen las aristas
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    return g

In [471]:
def Dorogovtsev(n,p):
    nombre_archivo = "Dorogovtsev_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan 3 nodos al grafo
    for i in range(3):
        g.agregar_nodo(i)
    # Se conectan los nodos agregados formando un triángulo
    g.agregar_arista("1", "2")
    g.agregar_arista("2", "3")
    g.agregar_arista("3", "1")
    # Se agregan los n-3 nodos restantes
    for i in range(4, n):
        lista = [] # Se guardan los nodos previamente conectados para evitar que se dupliquen las aristas
        g.agregar_nodo((i)) # Se agrega un nuevo nodo
        for k in range(4):
            j = random.choice(list(g.nodos.keys())) # Se escoge el nodo destino de forma aleatoria de las aristas existentes
            cond = j in lista # Se determina si el nodo se encuentra en el bucle actual para evitar repeticiones
            if i != j and cond == False: # Se omiten las conexiones hacia un mismo nodo y las aristas repetidas
                g.agregar_arista((i), (j))
            lista.append(j) # Se guardan los nodos actualmente conectados para evitar que se dupliquen las aristas
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    return g

In [472]:
# Erdos
# Se generan tres grafos por cada metodo con 30, 100 y 500 nodos
grafo1 = randomErdos(30,30,15)
print(grafo1)
grafo2 = randomErdos(100,100,50)
print(grafo2)
grafo3 = randomErdos(500,500,250)
print(grafo3)

digraph G {
	15 -> 6;
	6 -> 29;
	29 -> 14;
}
digraph G {
	29 -> 14;
	6 -> 29;
	15 -> 6;
}
digraph G {
	9 -> 28;
	9 -> 5;
	9 -> 13;
	23 -> 5;
	24 -> 0;
	29 -> 14;
	28 -> 20;
	28 -> 14;
	4 -> 21;
	25 -> 9;
	25 -> 5;
	17 -> 28;
	17 -> 19;
	5 -> 18;
	6 -> 29;
	1 -> 7;
	22 -> 15;
	20 -> 21;
	20 -> 0;
	13 -> 9;
	13 -> 29;
	13 -> 0;
	13 -> 6;
	11 -> 16;
	12 -> 10;
	12 -> 21;
	15 -> 6;
	26 -> 7;
	26 -> 5;
	0 -> 15;
}
digraph G {
	50 -> 17;
	50 -> 13;
	17 -> 73;
	73 -> 32;
	32 -> 75;
	75 -> 92;
	75 -> 41;
	41 -> 28;
	28 -> 9;
	9 -> 42;
}
digraph G {
	75 -> 92;
	75 -> 41;
	9 -> 42;
	28 -> 9;
	41 -> 28;
	32 -> 75;
	73 -> 32;
	17 -> 73;
	50 -> 17;
	50 -> 13;
}
digraph G {
	7 -> 31;
	7 -> 71;
	63 -> 62;
	31 -> 44;
	36 -> 88;
	74 -> 1;
	88 -> 51;
	88 -> 21;
	49 -> 77;
	83 -> 70;
	83 -> 73;
	51 -> 33;
	51 -> 55;
	73 -> 32;
	47 -> 53;
	82 -> 81;
	60 -> 88;
	60 -> 50;
	98 -> 39;
	98 -> 99;
	37 -> 14;
	30 -> 24;
	30 -> 89;
	57 -> 5;
	57 -> 73;
	57 -> 63;
	57 -> 50;
	0 -> 50;
	20 -> 66;
	20 -> 8;
	84 -> 

In [485]:
# Malla
# Se generan tres grafos por cada metodo con 30, 100 y 500 nodos
grafo4 = Malla(6,5,15)
print(grafo4)
grafo5 = Malla(10,10,50)
print(grafo5)
grafo6 = Malla(25,20,25)
print(grafo6)

digraph G {
	15 -> 16;
	15 -> 21;
	16 -> 17;
	21 -> 22;
	21 -> 27;
	17 -> 18;
	22 -> 23;
	27 -> 28;
	23 -> 24;
	28 -> 29;
	29 -> 30;
}
digraph G {
	17 -> 18;
	17 -> 23;
	23 -> 24;
	23 -> 29;
	29 -> 30;
	16 -> 17;
	16 -> 22;
	22 -> 28;
	15 -> 16;
	15 -> 21;
	21 -> 27;
}
digraph G {
	1 -> 2;
	1 -> 7;
	2 -> 3;
	2 -> 8;
	3 -> 4;
	3 -> 9;
	4 -> 5;
	4 -> 10;
	5 -> 6;
	5 -> 11;
	7 -> 8;
	7 -> 13;
	8 -> 9;
	8 -> 14;
	9 -> 10;
	9 -> 15;
	10 -> 11;
	10 -> 16;
	11 -> 12;
	11 -> 17;
	13 -> 14;
	13 -> 19;
	14 -> 15;
	14 -> 20;
	15 -> 16;
	15 -> 21;
	16 -> 17;
	16 -> 22;
	17 -> 18;
	17 -> 23;
	19 -> 20;
	19 -> 25;
	20 -> 21;
	20 -> 26;
	21 -> 22;
	21 -> 27;
	22 -> 23;
	22 -> 28;
	23 -> 24;
	23 -> 29;
	25 -> 26;
	26 -> 27;
	27 -> 28;
	28 -> 29;
	29 -> 30;
	31 -> 32;
	32 -> 33;
	33 -> 34;
	34 -> 35;
	35 -> 36;
}
digraph G {
	50 -> 51;
	50 -> 60;
	51 -> 52;
	60 -> 61;
	60 -> 70;
	52 -> 53;
	61 -> 62;
	70 -> 71;
	70 -> 80;
	53 -> 54;
	62 -> 63;
	71 -> 72;
	80 -> 81;
	80 -> 90;
	54 -> 55;
	63 -> 64;
	72 

In [478]:
# Geografico
# Se generan tres grafos por cada metodo con 30, 100 y 500 nodos
grafo7 = Geografico(30,10,25)
print(grafo7)
grafo8 = Geografico(100,10,50)
print(grafo8)
grafo9 = Geografico(500,10,250)
print(grafo9)

digraph G {
	25 -> 3;
	3 -> 26;
}
digraph G {
	3 -> 26;
	25 -> 3;
}
digraph G {
	0 -> 22;
	2 -> 13;
	3 -> 25;
	3 -> 26;
	10 -> 14;
	11 -> 27;
	13 -> 2;
	14 -> 10;
	22 -> 0;
	24 -> 29;
	25 -> 3;
	25 -> 26;
	26 -> 3;
	26 -> 25;
	27 -> 11;
	29 -> 24;
}
digraph G {
	50 -> 66;
	66 -> 61;
	61 -> 91;
}
digraph G {
	61 -> 91;
	66 -> 61;
	50 -> 66;
}
digraph G {
	0 -> 68;
	1 -> 32;
	1 -> 38;
	1 -> 85;
	2 -> 60;
	2 -> 75;
	3 -> 94;
	3 -> 99;
	4 -> 54;
	4 -> 90;
	5 -> 45;
	6 -> 28;
	9 -> 83;
	10 -> 20;
	15 -> 34;
	15 -> 83;
	16 -> 26;
	16 -> 77;
	16 -> 79;
	17 -> 96;
	20 -> 10;
	21 -> 55;
	23 -> 25;
	23 -> 68;
	23 -> 76;
	24 -> 38;
	24 -> 69;
	25 -> 23;
	25 -> 76;
	25 -> 92;
	26 -> 16;
	27 -> 88;
	28 -> 6;
	29 -> 44;
	30 -> 40;
	32 -> 1;
	32 -> 38;
	32 -> 85;
	33 -> 93;
	34 -> 15;
	34 -> 83;
	36 -> 52;
	38 -> 1;
	38 -> 24;
	38 -> 32;
	38 -> 69;
	38 -> 85;
	40 -> 30;
	44 -> 29;
	45 -> 5;
	47 -> 82;
	48 -> 60;
	48 -> 75;
	50 -> 66;
	52 -> 36;
	54 -> 4;
	54 -> 90;
	55 -> 21;
	60 -> 2;
	60 -> 48;
	60

In [479]:
# Gilbert
# Se generan tres grafos por cada metodo con 30, 100 y 500 nodos
grafo10 = Gilbert(30,0.2,12)
print(grafo10)
grafo11 = Gilbert(100,0.2,55)
print(grafo11)
grafo12 = Gilbert(500,0.2,200)
print(grafo12)

digraph G {
	12 -> 9;
	12 -> 18;
	18 -> 22;
	18 -> 0;
	18 -> 8;
	9 -> 2;
	9 -> 10;
	2 -> 15;
	15 -> 19;
	15 -> 20;
	0 -> 3;
	0 -> 1;
	0 -> 21;
	0 -> 26;
	0 -> 16;
	3 -> 6;
	6 -> 25;
	1 -> 13;
	21 -> 24;
	25 -> 17;
	26 -> 4;
	4 -> 7;
	17 -> 27;
	27 -> 14;
	27 -> 23;
	8 -> 5;
	16 -> 11;
	11 -> 28;
	28 -> 29;
}
digraph G {
	27 -> 20;
	27 -> 23;
	19 -> 27;
	7 -> 19;
	7 -> 26;
	21 -> 7;
	21 -> 29;
	28 -> 21;
	24 -> 28;
	8 -> 24;
	18 -> 8;
	22 -> 18;
	25 -> 22;
	14 -> 25;
	13 -> 14;
	17 -> 13;
	10 -> 17;
	0 -> 10;
	0 -> 16;
	16 -> 11;
	6 -> 0;
	4 -> 6;
	1 -> 4;
	15 -> 1;
	5 -> 15;
	3 -> 5;
	2 -> 3;
	9 -> 2;
	12 -> 9;
}
digraph G {
	0 -> 1;
	0 -> 2;
	0 -> 3;
	0 -> 10;
	0 -> 16;
	0 -> 17;
	0 -> 19;
	0 -> 20;
	0 -> 21;
	0 -> 22;
	0 -> 26;
	1 -> 4;
	1 -> 10;
	1 -> 13;
	1 -> 14;
	2 -> 3;
	2 -> 13;
	2 -> 15;
	2 -> 18;
	2 -> 21;
	2 -> 24;
	3 -> 2;
	3 -> 5;
	3 -> 6;
	3 -> 7;
	3 -> 11;
	3 -> 18;
	3 -> 27;
	4 -> 6;
	4 -> 7;
	4 -> 11;
	4 -> 14;
	4 -> 22;
	4 -> 24;
	4 -> 26;
	4 -> 29;
	5 -> 15;
	5 -> 17

digraph G {
	200 -> 2;
	200 -> 6;
	200 -> 25;
	200 -> 26;
	6 -> 23;
	23 -> 24;
	23 -> 28;
	2 -> 29;
	29 -> 31;
	29 -> 35;
	35 -> 41;
	35 -> 48;
	41 -> 44;
	41 -> 62;
	41 -> 68;
	41 -> 78;
	48 -> 87;
	48 -> 92;
	78 -> 88;
	78 -> 100;
	28 -> 90;
	68 -> 93;
	100 -> 105;
	100 -> 107;
	100 -> 113;
	87 -> 112;
	113 -> 127;
	113 -> 147;
	105 -> 137;
	127 -> 154;
	127 -> 166;
	147 -> 155;
	155 -> 156;
	154 -> 161;
	166 -> 170;
	170 -> 177;
	170 -> 179;
	170 -> 190;
	161 -> 182;
	161 -> 184;
	137 -> 187;
	190 -> 198;
	184 -> 199;
	198 -> 202;
	156 -> 203;
	203 -> 207;
	203 -> 212;
	187 -> 214;
	212 -> 227;
	227 -> 228;
	228 -> 232;
	232 -> 235;
	232 -> 243;
	232 -> 245;
	243 -> 251;
	202 -> 254;
	245 -> 266;
	245 -> 280;
	245 -> 285;
	266 -> 271;
	266 -> 272;
	266 -> 289;
	266 -> 309;
	271 -> 276;
	271 -> 286;
	289 -> 293;
	289 -> 314;
	289 -> 341;
	289 -> 362;
	285 -> 301;
	286 -> 310;
	254 -> 312;
	309 -> 318;
	312 -> 320;
	276 -> 336;
	310 -> 337;
	337 -> 339;
	318 -> 350;
	341 -> 357;
	339 

In [480]:
# Dorogovtsev
# Se generan tres grafos por cada metodo con 30, 100 y 500 nodos
grafo13 = Dorogovtsev(30,14)
print(grafo13)
grafo14 = Dorogovtsev(100,30)
print(grafo14)
grafo15 = Dorogovtsev(500,300)
print(grafo15)

digraph G {
	14 -> 7;
	14 -> 11;
	7 -> 4;
	7 -> 5;
	5 -> 0;
	5 -> 1;
	11 -> 9;
	11 -> 10;
	10 -> 2;
}
digraph G {
	4 -> 2;
	4 -> 0;
	7 -> 4;
	7 -> 5;
	5 -> 1;
	14 -> 7;
	14 -> 11;
	11 -> 9;
	11 -> 10;
}
digraph G {
	1 -> 2;
	2 -> 3;
	3 -> 1;
	4 -> 2;
	4 -> 0;
	5 -> 0;
	5 -> 1;
	5 -> 4;
	6 -> 1;
	6 -> 5;
	7 -> 4;
	7 -> 5;
	7 -> 0;
	8 -> 0;
	8 -> 2;
	8 -> 1;
	9 -> 2;
	9 -> 4;
	9 -> 5;
	10 -> 2;
	10 -> 4;
	10 -> 7;
	10 -> 9;
	11 -> 0;
	11 -> 9;
	11 -> 1;
	11 -> 10;
	12 -> 6;
	12 -> 11;
	12 -> 4;
	12 -> 7;
	13 -> 5;
	13 -> 10;
	13 -> 8;
	14 -> 7;
	14 -> 11;
	15 -> 6;
	15 -> 1;
	15 -> 4;
	15 -> 8;
	16 -> 2;
	16 -> 5;
	16 -> 8;
	17 -> 14;
	17 -> 4;
	18 -> 15;
	18 -> 9;
	18 -> 5;
	19 -> 10;
	19 -> 0;
	19 -> 15;
	19 -> 2;
	20 -> 16;
	20 -> 17;
	20 -> 0;
	20 -> 13;
	21 -> 12;
	21 -> 5;
	21 -> 17;
	22 -> 2;
	22 -> 19;
	22 -> 13;
	23 -> 15;
	23 -> 5;
	23 -> 11;
	23 -> 19;
	24 -> 7;
	24 -> 19;
	24 -> 11;
	24 -> 21;
	25 -> 17;
	25 -> 2;
	25 -> 20;
	25 -> 8;
	26 -> 2;
	26 -> 4;
	26 -> 22;
	26 -> 21;

In [481]:
# Barabasi
# Se generan tres grafos por cada metodo con 30, 100 y 500 nodos
grafo16 = Barabasi(30,10,20)
print(grafo16)
grafo17 = Barabasi(100,30,70)
print(grafo17)
grafo18 = Barabasi(500,150,150)
print(grafo18)

digraph G {
	20 -> 8;
	20 -> 18;
	8 -> 3;
	8 -> 6;
	18 -> 2;
	18 -> 16;
	18 -> 4;
	18 -> 10;
	18 -> 13;
	4 -> 1;
	16 -> 5;
	16 -> 7;
	10 -> 9;
	13 -> 12;
}
digraph G {
	8 -> 1;
	8 -> 2;
	8 -> 3;
	8 -> 4;
	8 -> 5;
	8 -> 6;
	8 -> 7;
	20 -> 8;
	20 -> 18;
	10 -> 9;
	18 -> 10;
	18 -> 13;
	18 -> 16;
	13 -> 12;
}
digraph G {
	2 -> 1;
	3 -> 1;
	3 -> 2;
	4 -> 1;
	4 -> 2;
	4 -> 3;
	5 -> 1;
	5 -> 2;
	5 -> 3;
	5 -> 4;
	6 -> 1;
	6 -> 2;
	6 -> 3;
	6 -> 4;
	6 -> 5;
	7 -> 1;
	7 -> 2;
	7 -> 3;
	7 -> 4;
	7 -> 5;
	7 -> 6;
	8 -> 1;
	8 -> 2;
	8 -> 3;
	8 -> 4;
	8 -> 5;
	8 -> 6;
	8 -> 7;
	9 -> 1;
	9 -> 2;
	9 -> 3;
	9 -> 4;
	9 -> 5;
	9 -> 6;
	9 -> 7;
	9 -> 8;
	10 -> 9;
	10 -> 6;
	10 -> 8;
	10 -> 3;
	10 -> 7;
	10 -> 2;
	11 -> 2;
	11 -> 9;
	11 -> 8;
	11 -> 5;
	11 -> 10;
	11 -> 4;
	12 -> 2;
	12 -> 5;
	12 -> 4;
	12 -> 10;
	12 -> 3;
	12 -> 6;
	12 -> 1;
	13 -> 5;
	13 -> 4;
	13 -> 6;
	13 -> 8;
	13 -> 7;
	13 -> 1;
	13 -> 12;
	14 -> 10;
	14 -> 1;
	14 -> 6;
	14 -> 5;
	14 -> 3;
	14 -> 9;
	14 -> 2;
	14 -> 8;
	15 -> 14;
	

digraph G {
	150 -> 14;
	150 -> 76;
	150 -> 119;
	150 -> 121;
	150 -> 126;
	150 -> 145;
	150 -> 148;
	119 -> 70;
	119 -> 81;
	119 -> 100;
	100 -> 8;
	100 -> 60;
	100 -> 69;
	69 -> 22;
	121 -> 11;
	121 -> 54;
	126 -> 106;
	126 -> 124;
	106 -> 105;
	124 -> 20;
	124 -> 115;
	20 -> 10;
	115 -> 74;
	74 -> 66;
	74 -> 67;
	66 -> 16;
	67 -> 42;
	145 -> 108;
	145 -> 136;
	108 -> 75;
	108 -> 92;
	92 -> 15;
	136 -> 13;
	148 -> 6;
	148 -> 21;
	148 -> 80;
	148 -> 134;
	148 -> 137;
	148 -> 146;
	148 -> 147;
	80 -> 49;
	134 -> 90;
	134 -> 128;
	137 -> 33;
	137 -> 46;
	137 -> 63;
	137 -> 103;
	103 -> 48;
	146 -> 24;
	146 -> 129;
	146 -> 144;
	129 -> 95;
	129 -> 97;
	97 -> 65;
	144 -> 116;
	144 -> 139;
	144 -> 141;
	139 -> 44;
	139 -> 62;
	139 -> 93;
	139 -> 120;
	139 -> 125;
	62 -> 38;
	93 -> 4;
	120 -> 85;
	85 -> 26;
	85 -> 35;
	141 -> 58;
	141 -> 83;
	141 -> 89;
	89 -> 5;
	89 -> 52;
	52 -> 39;
	147 -> 7;
	147 -> 122;
	147 -> 143;
	122 -> 73;
	73 -> 55;
	143 -> 32;
	143 -> 61;
	143 -> 82;
	143 -> 87;